In [1]:
import sys # aby importy z dołu działały
sys.path.append(r"D:\MyProjects_4Fun\projects\World of Warcraft\python-etl")

In [2]:
from sqlalchemy import text
#from moduly.db_admin import czerwony_przycisk
from moduly.db_core import utworz_engine_do_db, _czy_duplikat
from moduly.services_persist_wynik import przefiltruj_dane_misji, zapisz_misje_dialogi_ai_do_db
from sqlalchemy.exc import IntegrityError
from scraper_wiki_main import parsuj_misje_z_url
from moduly.ai_gemini import zaladuj_api_i_klienta
import json
import zlib
import base64
import pandas as pd
from pprint import pprint

In [3]:
klient = zaladuj_api_i_klienta(nazwa_api="API_TLUMACZENIE")

KLUCZ ZWARTY I GOTOWY!


In [4]:
silnik = utworz_engine_do_db()

In [ ]:
q_select_misje_id = text("""
    SELECT m.MISJA_ID_MOJE_PK
    FROM dbo.MISJE AS m
    WHERE m.KRAINA_EN = :kraina_en
      --AND m.MISJA_ID_Z_GRY IS NOT NULL
      AND m.MISJA_ID_Z_GRY <> 123456789
      AND m.MISJA_ID_MOJE_PK = 31
      AND NOT EXISTS (
          SELECT 1
          FROM dbo.MISJE_STATUSY AS ms
          WHERE ms.MISJA_ID_MOJE_FK = m.MISJA_ID_MOJE_PK
          AND ms.STATUS = N'1_PRZETŁUMACZONO'
    )
    ;
""")

q_select_tresc = text("""
WITH hashe AS (
    SELECT 
        m.MISJA_ID_MOJE_PK,
        z.HTML_SKOMPRESOWANY,
        ROW_NUMBER() OVER (
            PARTITION BY z.MISJA_ID_MOJE_FK 
            ORDER BY z.DATA_WYSCRAPOWANIA DESC
        ) AS r
    FROM dbo.ZRODLO AS z
    INNER JOIN dbo.MISJE AS m
      ON z.MISJA_ID_MOJE_FK = m.MISJA_ID_MOJE_PK
    WHERE 1=1
      AND m.KRAINA_EN = :kraina_en
      AND m.MISJA_ID_Z_GRY <> 123456789
      AND m.MISJA_ID_MOJE_PK = 31
)
SELECT 
    MISJA_ID_MOJE_PK, HTML_SKOMPRESOWANY
FROM hashe
WHERE r = 1
ORDER BY MISJA_ID_MOJE_PK
;
""")

q_select_npc = text("""
WITH wszystkie_idki AS (
    SELECT
        tabela_wartosci.ID_NPC
    FROM dbo.MISJE AS m
    CROSS APPLY (
        VALUES
            (m.NPC_START_ID),
            (m.NPC_KONIEC_ID)
    ) AS tabela_wartosci (ID_NPC)
    WHERE m.MISJA_ID_MOJE_PK = :misja_id

    UNION

    SELECT
        ds.NPC_ID_FK
    FROM dbo.DIALOGI_STATUSY AS ds
    WHERE ds.MISJA_ID_MOJE_FK = :misja_id
),

oczyszczone_dane AS (
    SELECT
        wi.ID_NPC,
        ns.STATUS,
        CASE
            WHEN CHARINDEX('[', ns.NAZWA) > 0
            THEN RTRIM(LEFT(ns.NAZWA, CHARINDEX('[', ns.NAZWA) - 1))
            ELSE ns.NAZWA
        END AS CZYSTA_NAZWA
    FROM wszystkie_idki AS wi
    INNER JOIN dbo.NPC_STATUSY AS ns
        ON wi.ID_NPC = ns.NPC_ID_FK
)

SELECT DISTINCT
    pvt.[0_ORYGINAŁ],
    pvt.[3_ZATWIERDZONO]
FROM oczyszczone_dane
PIVOT (
    MAX(CZYSTA_NAZWA)
    FOR STATUS IN ([0_ORYGINAŁ], [3_ZATWIERDZONO])
) AS pvt
;
""")

q_select_slowa_kluczowe = text("""
    SELECT 
        sk.SLOWO_EN,
        sk.SLOWO_PL
    FROM dbo.MISJE_SLOWA_KLUCZOWE AS msk
    INNER JOIN dbo.SLOWA_KLUCZOWE AS sk
    ON msk.SLOWO_ID = sk.SLOWO_ID_PK
    WHERE msk.MISJA_ID_MOJE_FK = :misja_id
""")

with silnik.connect() as conn:
    wyniki_misje_id = tuple(misja[0] for misja in conn.execute(q_select_misje_id, {"kraina_en": "Exile's Reach"}).all())
    #print(wyniki_misje_id)
    wyniki_z_bazy = conn.execute(q_select_tresc, {"kraina_en": "Exile's Reach"}).mappings().all()

    print(f"Znaleziono rekordów: {len(wyniki_z_bazy)}\n")
    
    for wiersz in wyniki_z_bazy:
        misja_id = wiersz["MISJA_ID_MOJE_PK"]
        zakodowane_dane = wiersz["HTML_SKOMPRESOWANY"]

        npc_z_bazy = conn.execute(q_select_npc, {"misja_id": misja_id}).all()
        slowa_kluczowe_z_bazy = conn.execute(q_select_slowa_kluczowe, {"misja_id": misja_id}).all()

        if not zakodowane_dane:
            print(f"SKIP [ID: {misja_id}] - Brak skompresowanego HTML w bazie.")
            continue

        try:
            skompresowane_bajty = base64.b64decode(zakodowane_dane)
            tekst_html = zlib.decompress(skompresowane_bajty).decode("utf-8")

            surowe_dane = parsuj_misje_z_url(None, html_content=tekst_html)
            #print(surowe_dane)
            
            przetworzone_dane = przefiltruj_dane_misji(surowe_dane, jezyk="EN")

            wsad_dla_geminisia_npc = set(npc for npc in npc_z_bazy)
            wsad_dla_geminisia_sk = set(slowo for slowo in slowa_kluczowe_z_bazy)

            wsad_dla_geminisia_cialo = json.dumps(przetworzone_dane, indent=4, ensure_ascii=False)

            lista_npc_tekst = "\n".join([f"- {n[0]} -> {n[1]}" for n in wsad_dla_geminisia_npc if n[0] and n[1]])
            lista_sk_tekst = "\n".join([f"- {k[0]} -> {k[1]}" for k in wsad_dla_geminisia_sk if k[0] and k[1]])
            #print(lista_sk_tekst)

            instrukcja = f"""
            Jesteś profesjonalnym tłumaczem fantasy specjalizującym się w grze World of Warcraft i powiązanych z tą marką dziełach (gry, książki, short stories, komiksy, wszystko). Twoim zadaniem jest przetłumaczenie danych misji (tytuły, cele, opisy, dialogi) z języka angielskiego na wysokiej jakości język polski, z zachowaniem ścisłych reguł formatowania danych.
            DANE WEJŚCIOWE: Otrzymasz słowniki nazw własnych oraz główny obiekt JSON z treścią misji.
            OBOWIĄZKOWE SŁOWNIKI NAZW WŁASNYCH: Podczas tłumaczenia musisz bezwzględnie stosować się do poniższych mapowań. Lista NPC (Angielski -> Polski): {lista_npc_tekst}
            Lista Słów Kluczowych (Angielski -> Polski): {lista_sk_tekst}

            ZASADY TŁUMACZENIA (STYL I TREŚĆ):
            Klimat i Styl: Zachowaj sens, emocje i ton oryginału. Tłumaczenie musi brzmieć naturalnie dla polskiego gracza fantasy (klimat World of Warcraft). Unikaj kalk językowych.
            Płynność: Unikaj sztywnego, dosłownego przekładu. Stwórz płynny, profesjonalny tekst literacki. Jeśli angielska konstrukcja brzmi topornie, wygładź ją w polszczyźnie, zachowując pierwotny sens.
            Wierność i Czystość: Nie dodawaj żadnych informacji od siebie, nie komentuj tekstu. Jeśli w oryginale jest literówka – skoryguj ją w tłumaczeniu po cichu.
            Placeholdery Techniczne: Zachowaj nienaruszone wszelkie znaczniki techniczne, takie jak: {{PLAYER_NAME}}, %s, %d, $n, $g, znaczniki kolorów (np. |cFFFF0000...|r) oraz tagi formatowania (, \n). Muszą znaleźć się w tłumaczeniu w odpowiednim miejscu.
            Nazwy Własne: Używaj nazw z podanych wyżej słowników. Jeśli nazwy nie ma w słowniku – zostaw ją w oryginale (chyba że jest to pospolite słowo możliwe do naturalnego przetłumaczenia, np. "Lake" -> "Jezioro", ale nazwy miast/krain zostawiasz, np. "Ironforge" no chyba, że podano tłumaczenie w słowniku).
            Kontekst: Pamiętaj, że teksty są ze sobą powiązane. Dialogi (gossipy/dymki) dotyczą tej konkretnej misji.

            ZASADY TECHNICZNE (STRUKTURA JSON I KLUCZE):
            Struktura: Zwrócony JSON musi mieć identyczną strukturę zagnieżdżenia jak oryginał. Nie usuwaj żadnych obiektów.
            Podmiana Kluczy Językowych:
            Wszędzie tam, gdzie klucz kończy się na "_EN" (np. "Misje_EN", "Treść_EN", "npc_en"), zmień końcówkę na "_PL" (np. "Misje_PL", "Treść_PL", "npc_pl").
            Zachowaj wielkość liter przedrostka (np. "npc_en" -> "npc_pl", ale "NPC_EN" -> "NPC_PL").
            Klucze bez sufiksu językowego (np. "id", "typ", "nr_bloku") pozostaw BEZ ZMIAN.
            Puste Pola: Jeśli jakakolwiek sekcja, lista lub pole w oryginale jest puste (np. "Gossipy_Dymki_EN": [] lub "TRESC": ""), w wynikowym JSONie musi pozostać puste (z odpowiednio zmienioną nazwą klucza na _PL). Nie usuwaj pustych kluczy.
            Kolejność i ID: Każdy element list (np. dialogi) musi zostać zwrócony w tej samej kolejności i z tym samym ID, co w oryginale.
            Format Wyjściowy: Zwróć tylko i wyłącznie poprawny kod JSON. Bez bloków markdown (```json), bez komentarzy wstępnych czy końcowych.
            """

            try:
                odpowiedz = klient.models.generate_content(
                    model="gemini-3-pro-preview",
                    contents=wsad_dla_geminisia_cialo,
                    config={
                        "system_instruction": instrukcja,
                        "response_mime_type": "application/json"
                    }
                )

                przetlumaczone = json.loads(odpowiedz.text)
                
                zapisz_misje_dialogi_ai_do_db(
                    silnik=silnik, 
                    misja_id=misja_id, 
                    przetlumaczone=przetlumaczone, 
                    status="1_PRZETŁUMACZONO"
                )

            except Exception as er:
                print(f"BŁĄD w tłumaczeniu/zapisie misji: {misja_id} --> {er}")

        except Exception as e:
            print(f"BŁĄD ogólny przy ID {misja_id}: {e}")
# rozsplitowac na kilka funkcji: jedna do tlumaczenia, druga do zapisu

Znaleziono rekordów: 1


--- [START] Zapis misji ID: 31 | Status: 1_PRZETŁUMACZONO ---
-> Przygotowano danych misji: 14 wierszy.
-> Zaktualizowano tytuł na: 'Pakt czarnoksiężnika'
-> Rozpoczynam mapowanie NPC w dialogach...
-> Przygotowano dialogów: 5 wierszy.
-> COMMIT: Dane zostały wysłane do bazy.
--- [KONIEC] Sukces dla misji ID: 31 ---

